---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [437]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [438]:
df.shape

(500,)

In [439]:
import re
lst="\n04/20/2009\n04/20/09\n4/20/09\n4/3/09\nMar-20-2009\nMar 20, 2009\nMarch 20, 2009\nMar. 20\n2009\nMar 20 2009\n20 Mar 2009\n20 March 2009\n20 Mar. 2009\n20 March\n2009\nMar 20th, 2009\nMar 21st\n2009\nMar 22nd\n2009\nFeb 2009\nSep 2009\nOct 2010\n6/2008\n12/2009\n2009\n2010\n"

In [440]:


# #Test case
# df = pd.Series(['04/20/2009', '04/20/09', '4/20/09', '04-20-09', '12-12-2015', '4/3/09', 'Mar-20-2009', 'Mar 20, 2009', 'March 20, 2009', 'Mar. 20, 2009', 'Mar 20 2009', '20 Mar 2009', '20 March 2009', '20 Mar. 2009', '20 March, 2009', 'Mar 20th, 2009', 'Mar 21st, 2009', 'Mar 22nd, 2009', 'Feb 2009', 'Sep 2009', 'Oct. 2010', 'November 2015', '6/2008', '12/2009', '2009', '2010', '2June, 1999'])
# df


In [441]:
df1=df.str.extractall(r'(?P<Datetime>(?P<Month>\d?\d)[/-](?P<Day>\d?\d)[/-](?P<Year>\d{2}|\d{4})\b)')
month_dict = dict({'September': 9, 'Mar': 3, 'November': 11, 'Jul': 7, 'January': 1, 'December': 12,
                   'Feb': 2, 'May': 5, 'Aug': 8, 'Jun': 6, 'Sep': 9, 'Oct': 10, 'June': 6, 'March': 3,
                   'February': 2, 'Dec': 12, 'Apr': 4, 'Jan': 1, 'Janaury': 1,'August': 8, 'October': 10,
                   'July': 7, 'Since': 1, 'Nov': 11, 'April': 4, 'Decemeber': 12, 'Age': 8})
df1['Year']=df1['Year'].map(lambda x: x if len(x)==4 else int('19'+str(x)))

In [442]:
df2=df.str.extractall(r'(?P<Datetime>(?P<Month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ -\.]+(?P<Day>\d\d)[a-z]*[ ,-]+(?P<Year>\d{4})\b)')
df2['Month']=df2['Month'].map(lambda x: month_dict[x])

In [443]:
df3=df.str.extractall(r'(?P<Datetime>(?P<Day>\d?\d)?[ ,-]*(?P<Month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ ,\.]+(?P<Year>\d{4})\b)')
df3['Month']=df3['Month'].map(lambda x: month_dict[x])
df3.fillna(1, inplace=True)

In [444]:
df1=df1.append(df2)
df1=df1.append(df3)

In [445]:

df4=df.str.extractall(r'(?P<Datetime>\b(?P<Month>\d{1,2})[/](?P<Year>\d{4})\b)')
df4['Day']=1
df4
idx=[i[0] for i in df4.index]
df4.index=idx
idx_total=[i[0] for i in df1.index]
df1.index=idx_total
idx_drop=[i for i in idx if i in idx_total]
df4.drop(idx_drop,inplace=True)
df1=df1.append(df4)

In [446]:
df1.index
#df4.index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            445, 446, 447, 448, 449, 450, 451, 452, 453, 454],
           dtype='int64', length=453)

In [447]:
df5=df.str.extractall(r'(?P<Datetime>(?P<Year>\d{4})\b)')
df5
idx=[i[0] for i in df5.index]
df5.index=idx
idx_total=df1.index
idx_drop=[i for i in idx if i in idx_total]
df5.drop(idx_drop,inplace=True)
df5['Day']=1
df5['Month']=1
df1=df1.append(df5)

In [448]:
[i for i in df.index if i not in df1.index]

[]

In [449]:
df1=df1.sort_values(['Year','Month','Day'])
df1['true_index']=df1.index

In [450]:
df6=pd.DataFrame(df1['true_index'])
df6['position']=range(df1.shape[0])

In [451]:
df6

,true_index,position
9,9,0
84,84,1
53,53,2
2,2,3
28,28,4
13,13,5
98,98,6
111,111,7
31,31,8
57,57,9


In [458]:
def date_sorter():
    df1=df.str.extractall(r'(?P<Datetime>(?P<Month>\d?\d)[/-](?P<Day>\d?\d)[/-](?P<Year>\d{2}|\d{4})\b)')
    month_dict = dict({'September': 9, 'Mar': 3, 'November': 11, 'Jul': 7, 'January': 1, 'December': 12,
                       'Feb': 2, 'May': 5, 'Aug': 8, 'Jun': 6, 'Sep': 9, 'Oct': 10, 'June': 6, 'March': 3,
                       'February': 2, 'Dec': 12, 'Apr': 4, 'Jan': 1, 'Janaury': 1,'August': 8, 'October': 10,
                       'July': 7, 'Since': 1, 'Nov': 11, 'April': 4, 'Decemeber': 12, 'Age': 8})
    df1['Year']=df1['Year'].map(lambda x: x if len(x)==4 else int('19'+str(x)))

    df2=df.str.extractall(r'(?P<Datetime>(?P<Month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ -\.]+(?P<Day>\d\d)[a-z]*[ ,-]+(?P<Year>\d{4})\b)')
    df2['Month']=df2['Month'].map(lambda x: month_dict[x])

    df3=df.str.extractall(r'(?P<Datetime>(?P<Day>\d?\d)?[ ,-]*(?P<Month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ ,\.]+(?P<Year>\d{4})\b)')
    df3['Month']=df3['Month'].map(lambda x: month_dict[x])
    df3.fillna(1, inplace=True)

    df1=df1.append(df2)
    df1=df1.append(df3)

    df4=df.str.extractall(r'(?P<Datetime>\b(?P<Month>\d{1,2})[/](?P<Year>\d{4})\b)')
    df4['Day']=1
    df4
    idx=[i[0] for i in df4.index]
    df4.index=idx
    idx_total=[i[0] for i in df1.index]
    df1.index=idx_total
    idx_drop=[i for i in idx if i in idx_total]
    df4.drop(idx_drop,inplace=True)
    df1=df1.append(df4)

    df5=df.str.extractall(r'(?P<Datetime>(?P<Year>\d{4})\b)')
    df5
    idx=[i[0] for i in df5.index]
    df5.index=idx
    idx_total=df1.index
    idx_drop=[i for i in idx if i in idx_total]
    df5.drop(idx_drop,inplace=True)
    df5['Day']=1
    df5['Month']=1
    df1=df1.append(df5)
    df1['true_index']=df1.index
    df1['Year']=df1['Year'].apply(int)
    df1['Month']=df1['Month'].apply(int)
    df1['Day']=df1['Day'].apply(int)
    
    df1=df1.sort_values(['Year','Month','Day'])
    df1.index=range(df1.shape[0])
    # Your code here
    
    return pd.Series(df1['true_index'])# Your answer here

In [459]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Name: true_index, Length: 500, dtype: int64

In [429]:
# df1,df2=date_sorter(df)
# df1.to_csv('df1.csv',index=False)